In [1]:
import sys, os
from pprint import pprint

from app_store_scraper import AppStore, Podcast
import numpy as np
import pandas as pd
import serpapi
import json

In [ ]:
minecraft = AppStore(country="nz", app_name="minecraft")
minecraft.review(how_many=20)

pprint(minecraft.reviews)
pprint(minecraft.reviews_count)

In [10]:
slack = AppStore(country='us', app_name='slack', app_id = '618783545')

slack.review(how_many=2000)

2023-12-23 22:54:01,167 [INFO] Base - Initialised: AppStore('us', 'slack', 618783545)
2023-12-23 22:54:01,168 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/slack/id618783545
2023-12-23 22:54:06,230 [INFO] Base - [id:618783545] Fetched 360 reviews (360 fetched in total)
2023-12-23 22:54:11,753 [INFO] Base - [id:618783545] Fetched 760 reviews (760 fetched in total)
2023-12-23 22:54:17,090 [INFO] Base - [id:618783545] Fetched 1120 reviews (1120 fetched in total)
2023-12-23 22:54:22,393 [INFO] Base - [id:618783545] Fetched 1500 reviews (1500 fetched in total)
2023-12-23 22:54:27,874 [INFO] Base - [id:618783545] Fetched 1900 reviews (1900 fetched in total)
2023-12-23 22:54:29,248 [INFO] Base - [id:618783545] Fetched 2000 reviews (2000 fetched in total)


In [19]:
slackdf = pd.DataFrame(np.array(slack.reviews),columns=['review'])
slackdf2 = slackdf.join(pd.DataFrame(slackdf.pop('review').tolist()))
slackdf2.head()

,date,developerResponse,review,rating,isEdited,userName,title
0,2021-11-19 15:37:47,"{'id': 26382584, 'body': 'Hey there, thank you...",The app is easy to learn and has become an int...,4,False,Hippiegal23,A few months in. Mostly good.
1,2022-04-14 14:50:10,NaN,The recent iPad App update misses the mark qui...,2,False,RufHaus71,"One Step Forward, Two Steps Back for iPad"
2,2022-10-31 09:26:51,"{'id': 32912428, 'body': 'Thanks for your feed...","When resizing the window of Slack, the changes...",3,False,dacloo,Does not work well with iPadOS 16
3,2021-06-16 23:48:53,NaN,I love slack. I started using slack in January...,5,False,AnC.Really?!,Workplace Communications
4,2022-08-05 01:50:27,"{'id': 31278707, 'body': 'Hi there, We’d be in...",Post-acquisition the Slack iOS app has taken s...,2,True,Just Your Usual Reviewer,Getting worse and worse


## Using Google Search API

In [2]:
serp_api_key = os.environ['SERP_API_KEY']
print(serp_api_key)

6fa6456459ea3414f9aa487e321b06e580aaf1414b30cb068f328e0d4f75273b


In [6]:
search_terms = ['diabetes']
engines = ['apple_app_store', 'google_play']
countries = ['de']
languages = ['de-de']
num = 20

In [ ]:
params_list = []

for term in search_terms:
    

In [3]:
params = {
    'api_key': serp_api_key,        # https://serpapi.com/manage-api-key
    'engine': 'apple_app_store',    # SerpApi search engine	
    'term': 'diabetes',         # search query
    'device': 'desktop',            # device to get the results
    'country': 'de',                # country for the search
    'lang': 'de-de',                # language for the search
    'disallow_explicit': False,     # disallowing explicit apps
    'num': 20,                      # number of items per page
    'page': 0,                      # pagination
    # 'property': 'developer'       # developer of an app
}

In [4]:
client = serpapi.Client(api_key=serp_api_key)
app_store_results = []

while True:
    search = client.search(params)
    new_page_results = search.as_dict()     # JSON -> Python dict
    
    app_store_results.extend(new_page_results['organic_results'])

    if 'next' in new_page_results.get('serpapi_pagination', {}):
        params['page'] += 1
    else:
        break

In [5]:
#print(json.dumps(app_store_results, indent=4, ensure_ascii=False))
output_fp = './appstore_results.json'
with open(output_fp, 'w') as f:
    f.write(json.dumps(app_store_results, indent=4, ensure_ascii=False))